# Sum Frame Sudoku
---
_**Source**: Slides "Constraint programming 1 - Modelling with OR-Tools"_  
_**Author**: Adrian Kauz_

>Fill in numbers from 1 to 9 so that each row, column and 3x3 block contains each number exactly once. Numbers in the outside frame equal the sum of the first three numbers in the corresponding row or column in the given direction.

![MagicSquares](images/SumFrameSudoku.png)

## Imports

In [1]:
from ortools.constraint_solver import pywrapcp
import itertools
import numpy as np

## Helper functions

In [2]:
def printSolution(currentSolution):
    # Top Line
    print("╔═══════════╦═══════════╦═══════════╗")
    
    # Field
    for row in range(9):
        newRow = "║ "
        
        for column in range(9):
            newRow += "{}".format(currentSolution[row][column].Value())
            newRow += " ║ " if (column + 1) % 3 == 0 else " │ "            
                                  
        print(newRow)
        
        if((row == 2) or (row == 5)):
            print("╠═══════════╬═══════════╬═══════════╣")
        else:
            if (row != 8):
                print("║───┼───┼───║───┼───┼───║───┼───┼───║");
        
    # Bottom Line
    print("╚═══════════╩═══════════╩═══════════╝\n")

## Solution

In [3]:
solver = pywrapcp.Solver("MagicSquares")

# n x n Matrix of Decision Variables
board = [[solver.IntVar(1, 9) for _j in range(9)] for _i in range(9)]

# Numbers of the outside frame of the Sum Frame Sudoku
# Values are read from left to right or from top to bottom
topFrame    = [15, 18, 12, 11, 21, 13, 15, 17, 13];
bottomFrame = [15, 9, 21, 10, 16, 19, 13, 15, 17];
leftFrame   = [8, 15, 22, 11, 13, 21, 18, 19, 8];
rightFrame  = [22, 8, 15, 22, 12, 11, 15, 13, 17];


# --------------------------------------------------------------------------------
# Set constraints
# --------------------------------------------------------------------------------
# Each row/column contains only different values:
for i in range(9):
    solver.Add(solver.AllDifferent([board[i][j] for j in range(9)])) # Rows
    solver.Add(solver.AllDifferent([board[j][i] for j in range(9)])) # Columns

# Tuples (i, j) for all cell indices:
cell_indices = list(itertools.product(range(3), repeat=2))

# Every 3x3 cell contains only different values
for i, j in cell_indices:
    solver.Add(solver.AllDifferent([board[i * 3 + di] [j * 3 + dj] for di in range(3) for dj in range(3)]))

# Now feed the solver with the outside frame number
for x in range(9):
    # Rows
    solver.Add(solver.Sum([board[x][0], board[x][1], board[x][2]]) == leftFrame[x])
    solver.Add(solver.Sum([board[x][6], board[x][7], board[x][8]]) == rightFrame[x])

    # Columns
    solver.Add(solver.Sum([board[0][x], board[1][x], board[2][x]]) == topFrame[x])
    solver.Add(solver.Sum([board[6][x], board[7][x], board[8][x]]) == bottomFrame[x])


# --------------------------------------------------------------------------------
# Now solve it!
# --------------------------------------------------------------------------------
db = solver.Phase(list(np.concatenate(board)), # Sudoku board as array
                solver.INT_VAR_SIMPLE,         # Variable selection policy for search
                solver.INT_VALUE_SIMPLE)       # Value selection policy for search

solver.NewSearch(db)

if solver.NextSolution():
    printSolution(board)
    
solver.EndSearch()

#Display the number of solutions found:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime: {}ms".format(solver.WallTime()))
print("Failures: {}".format(solver.Failures()))
print("Branches: {}".format(solver.Branches()))

╔═══════════╦═══════════╦═══════════╗
║ 1 │ 4 │ 3 ║ 2 │ 8 │ 5 ║ 9 │ 6 │ 7 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 8 │ 5 │ 2 ║ 6 │ 9 │ 7 ║ 4 │ 3 │ 1 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 6 │ 9 │ 7 ║ 3 │ 4 │ 1 ║ 2 │ 8 │ 5 ║ 
╠═══════════╬═══════════╬═══════════╣
║ 2 │ 3 │ 6 ║ 7 │ 1 │ 4 ║ 8 │ 5 │ 9 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 4 │ 8 │ 1 ║ 9 │ 5 │ 6 ║ 3 │ 7 │ 2 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 9 │ 7 │ 5 ║ 8 │ 2 │ 3 ║ 6 │ 1 │ 4 ║ 
╠═══════════╬═══════════╬═══════════╣
║ 7 │ 2 │ 9 ║ 1 │ 3 │ 8 ║ 5 │ 4 │ 6 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 5 │ 6 │ 8 ║ 4 │ 7 │ 2 ║ 1 │ 9 │ 3 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 3 │ 1 │ 4 ║ 5 │ 6 │ 9 ║ 7 │ 2 │ 8 ║ 
╚═══════════╩═══════════╩═══════════╝

Solutions: 1
Runtime: 5ms
Failures: 40
Branches: 87
